In [1]:
import pandas as pd

## 1. Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы

In [2]:
# Загружаем данные по рейтингам
ratings = pd.read_csv('u.data.csv', sep='\t', names=['user_id','item_id','rating','timestamp'])

ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
# Загружаем данные по фильмам
movies = pd.read_csv('u.item.csv', sep='|', \
                     names=["movie_id", "movie_title", "release_date", "video_release_date", "IMDb_URL", "unknown", \
                            "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", \
                            "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", \
                            "War", "Western"], encoding='latin-1')

movies.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## 2. Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок

In [4]:
# При помощи группировки по user_id, считаем общее количество вхождений, сортируем по убыванию и выводим первый элемент.
top_rater = ratings.groupby('user_id')['user_id'].count().sort_values(ascending=False).index[0]
print(f'Id пользователя поставившего больше всего оценок: {top_rater}')

Id пользователя поставившего больше всего оценок: 405


## 3. Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь

In [5]:
# Создаем датафрейм, имеющий сведения только по нашему пользователю. Применяем простой фильтр, 
# сравнивая занчения колонки с искомым
user_405 = ratings[ratings['user_id'] == top_rater]
user_405.head()

,user_id,item_id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447


## 4. Добавьте к датафрейму из задания 3 столбцы:
По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет


In [6]:
# Удаляем из датасета фильмов данные (по колонкам), которые нам точно не нужны
movies.drop(['movie_title', 'video_release_date', 'IMDb_URL'], axis=1, inplace=True)

In [7]:
# Объединяем наши датасеты. Далее чистим дату, оставляя только год. Переименовываем колонку с датой, на год.
merged = user_405.merge(movies, how='left', left_on='item_id', right_on='movie_id')
merged['release_date'] = merged['release_date'].apply(lambda x: x.split('-')[-1])
merged.rename(columns={'release_date':'release_year'}, inplace=True)
merged.head()

,user_id,item_id,rating,timestamp,movie_id,release_year,unknown,Action,Adventure,Animation,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,885544911,56,1994,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,405,592,1,885548670,592,1995,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,405,1582,1,885548670,1582,1947,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,405,171,1,885549544,171,1991,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,405,580,1,885547447,580,1995,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


Cтолбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей

In [8]:
# Создаем 2 вспомогательных датафрейма - с общим количеством оценок и их суммой. Переименовываем рейтинг в целевые значения.
# Объединяем датафреймы.
total_rates_count = ratings.groupby('item_id')['rating'].count().reset_index()
total_rates_count.rename(columns={'rating':'rates_count'}, inplace=True)

total_rates = ratings.groupby('item_id')['rating'].sum().reset_index()
total_rates.rename(columns={'rating':'rates_sum'}, inplace=True)

count_sum_rates = total_rates_count.merge(total_rates, how='left')
count_sum_rates.head()

,item_id,rates_count,rates_sum
0,1,452,1753
1,2,131,420
2,3,90,273
3,4,209,742
4,5,86,284


In [9]:
# Объединяем вспомогательный датафрейм с рейтингами с нашим предобработанным. Читсим ненудные данные. 
# Смотрим, все ли типы данных в порядке.
total_df = merged.merge(count_sum_rates, how='left')
total_df.drop(['user_id', 'item_id', 'timestamp', 'movie_id'], axis=1, inplace=True)
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 0 to 736
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rating        737 non-null    int64 
 1   release_year  737 non-null    object
 2   unknown       737 non-null    int64 
 3   Action        737 non-null    int64 
 4   Adventure     737 non-null    int64 
 5   Animation     737 non-null    int64 
 6   Children's    737 non-null    int64 
 7   Comedy        737 non-null    int64 
 8   Crime         737 non-null    int64 
 9   Documentary   737 non-null    int64 
 10  Drama         737 non-null    int64 
 11  Fantasy       737 non-null    int64 
 12  Film-Noir     737 non-null    int64 
 13  Horror        737 non-null    int64 
 14  Musical       737 non-null    int64 
 15  Mystery       737 non-null    int64 
 16  Romance       737 non-null    int64 
 17  Sci-Fi        737 non-null    int64 
 18  Thriller      737 non-null    int64 
 19  War     

In [10]:
# Приводим все типы к int32, большИх значений нет и все целочисленные. 32 резервирует меньше памяти.
total_df = total_df.astype(int)
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 0 to 736
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   rating        737 non-null    int32
 1   release_year  737 non-null    int32
 2   unknown       737 non-null    int32
 3   Action        737 non-null    int32
 4   Adventure     737 non-null    int32
 5   Animation     737 non-null    int32
 6   Children's    737 non-null    int32
 7   Comedy        737 non-null    int32
 8   Crime         737 non-null    int32
 9   Documentary   737 non-null    int32
 10  Drama         737 non-null    int32
 11  Fantasy       737 non-null    int32
 12  Film-Noir     737 non-null    int32
 13  Horror        737 non-null    int32
 14  Musical       737 non-null    int32
 15  Mystery       737 non-null    int32
 16  Romance       737 non-null    int32
 17  Sci-Fi        737 non-null    int32
 18  Thriller      737 non-null    int32
 19  War           737 non-null   

## 5. Сформируйте X_train, X_test, y_train, y_test



In [11]:
# Подключаем библиотеки.
from sklearn.model_selection import train_test_split

In [12]:
# Для удобства выводим все имеющиеся колонки в датафрейме
total_df.columns

Index(['rating', 'release_year', 'unknown', 'Action', 'Adventure', 'Animation',
       'Children's', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
       'Thriller', 'War', 'Western', 'rates_count', 'rates_sum'],
      dtype='object')

In [13]:
# Объявляем признаки и "искомое"
X = total_df[["release_year", "unknown", "Action", "Adventure", "Animation",
       "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
       "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
       "Thriller", "War", "Western", "rates_count", "rates_sum"]]
y = total_df['rating']

In [14]:
# Формируем X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 38)

## 6. Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах


In [15]:
# Подключаем библиотеки.
from sklearn.linear_model import LinearRegression

In [16]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## 7. Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [17]:
# Подключаем библиотеки для оценки среднеквадратичной ошибки и считаем ее.
from sklearn.metrics import mean_squared_error
  
mean_squared_error(y_test, y_pred)

"""
Как точно интерпретировать значение я не знаю, но чем ближе к 0, тем выше точность модели. 
У нас явно не очень хороший показатель.
"""

'\nКак точно интерпретировать значение я не знаю, но чем ближе к 0, тем выше точность модели. \nУ нас явно не очень хороший показатель.\n'

## 8. Загрузить данные в spark

In [18]:
# Подключаем библиотеки.
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [19]:
# Загружаем данные по рейтингам
ratings_spark = spark.read.csv('u.data.csv', sep='\t', inferSchema=True)
ratings_spark = ratings_spark.withColumnRenamed('_c0', 'user_id')\
            .withColumnRenamed('_c1', 'item_id')\
            .withColumnRenamed('_c2', 'rating')\
            .withColumnRenamed('_c3', 'timestamp')

ratings_spark.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [75]:
# Загружаем данные по фильмам
movies_spark = spark.read.csv('u.item.csv', sep='|', inferSchema=True)
movies_spark.show(5)

+---+-----------------+-----------+----+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|_c0|              _c1|        _c2| _c3|                 _c4|_c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|
+---+-----------------+-----------+----+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|  1| Toy Story (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  0|  0|  1|  1|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|
|  2| GoldenEye (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  1|  1|  0|  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   0|   0|
|  3|Four Rooms (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  0|  0|  0|  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   0|   0|
|  4|Get Shorty (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  1|  0|  0|  0|   1|  

## 9. Средствами спарка вывести среднюю оценку для каждого фильма



In [21]:
# Группируем по фильму и применяем функцию среднего по оценкам
avg_rate = ratings_spark.groupby('item_id').avg('rating')
avg_rate.show(5)

+-------+------------------+
|item_id|       avg(rating)|
+-------+------------------+
|    496| 4.121212121212121|
|    471|3.6108597285067874|
|    463| 3.859154929577465|
|    148|          3.203125|
|   1342|               2.5|
+-------+------------------+
only showing top 5 rows



## 10. Посчитайте средствами спарка среднюю оценку для каждого жанра

In [76]:
# Добавил заголовки
movies_spark = movies_spark.selectExpr("_c0 as movie_id", "_c1 as movie_title" , "_c2 as release_date",\
                                       "_c3 as video_release_date", "_c4 as IMDb_URL", "_c5 as unknown", "_c6 as Action",\
                                       "_c7 as Adventure", "_c8 as Animation", "_c9 as Childrens", "_c10 as Comedy",\
                                       "_c11 as Crime" , "_c12 as Documentary", "_c13 as Drama", "_c14 as Fantasy",\
                                       "_c15 as Film_Noir", "_c16 as Horror", "_c17 as Musical", "_c18 as Mystery",\
                                       "_c19 as Romance", "_c20 as Sci_Fi", "_c21 as Thriller" , "_c22 as War",\
                                       "_c23 as Western")

movies_spark.show(5)

+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|      movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film_Noir|Horror|Musical|Mystery|Romance|Sci_Fi|Thriller|War|Western|
+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1| Toy Story (1995)| 01-Jan-1995|              NULL|http://us.imdb.co...|      0|     0|        0|        1|        1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2| GoldenEye (1995)| 01-Jan-1995|              NULL|

In [77]:
# Объеденим наши датафреймы c рейтингами и оценками
joined = ratings_spark.join(movies_spark, ratings_spark.item_id == movies_spark.movie_id, 'inner')
joined.show(1)

+-------+-------+------+---------+--------+------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|user_id|item_id|rating|timestamp|movie_id| movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film_Noir|Horror|Musical|Mystery|Romance|Sci_Fi|Thriller|War|Western|
+-------+-------+------+---------+--------+------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|    196|    242|     3|881250949|     242|Kolya (1996)| 24-Jan-1997|              NULL|http://us.imdb.co...|      0|     0|        0|        0|        0|     1|    0|          0|    0|      0|        0|     0| 

In [78]:
# Уберем лишние столбцы.
columns_to_drop = ['movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'user_id', 'item_id', 'timestamp']
joined = joined.drop(*columns_to_drop)
joined.show(2)

+------+--------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|rating|movie_id|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film_Noir|Horror|Musical|Mystery|Romance|Sci_Fi|Thriller|War|Western|
+------+--------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|     3|     242|      0|     0|        0|        0|        0|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|     3|     302|      0|     0|        0|        0|        0|     0|    1|          0|    0|      0|        1|     0|      0|      1|      0|     0|       1|  0|      0|
+------+--------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+---

In [79]:
#Словарь жанров
genres = joined.columns
genres = genres[2:]
genres

['unknown',
 'Action',
 'Adventure',
 'Animation',
 'Childrens',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film_Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci_Fi',
 'Thriller',
 'War',
 'Western']

In [103]:
# Через цикл прогоняем по каждому жанру из нашего списка, по условию жанра и группируя по жанру считаем средний рейтинг.

for genre in genres:
    df_filter = joined.where(joined[genre] == 1)
    avg_rate = df_filter.groupby(genre).avg('rating')
    avg_rate.show()

+-------+-----------+
|unknown|avg(rating)|
+-------+-----------+
|      1|        3.2|
+-------+-----------+

+------+-----------------+
|Action|      avg(rating)|
+------+-----------------+
|     1|3.480245417953027|
+------+-----------------+

+---------+-----------------+
|Adventure|      avg(rating)|
+---------+-----------------+
|        1|3.503526503308369|
+---------+-----------------+

+---------+------------------+
|Animation|       avg(rating)|
+---------+------------------+
|        1|3.5766990291262135|
+---------+------------------+

+---------+------------------+
|Childrens|       avg(rating)|
+---------+------------------+
|        1|3.3532442216652742|
+---------+------------------+

+------+------------------+
|Comedy|       avg(rating)|
+------+------------------+
|     1|3.3940734781442745|
+------+------------------+

+-----+------------------+
|Crime|       avg(rating)|
+-----+------------------+
|    1|3.6322780881440098|
+-----+------------------+

+-----------+

## 11. В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)

In [27]:
# Топ-5 фильмов по сумме оценок
# Взяли датафрейм, сгруппировали по фильмам, посчитали сумму рейтнигов по фильмам, 
# отсортировали по этой сумме в порядке убывания и вывели топ 5.

ratings_spark.groupby('item_id').sum('rating').sort('sum(rating)', ascending=False).show(5)

+-------+-----------+
|item_id|sum(rating)|
+-------+-----------+
|     50|       2541|
|    100|       2111|
|    181|       2032|
|    258|       1936|
|    174|       1786|
+-------+-----------+
only showing top 5 rows



In [28]:
# Топ-5 фильмов по количеству оценок
# Взяли датафрейм, сгруппировали по фильмам, посчитали количество проставленных оценок по фильмам, 
# отсортировали по вычисленной переменной в порядке убывания и вывели топ 5.

ratings_spark.groupby('item_id').count().sort('count', ascending=False).show(5)

+-------+-----+
|item_id|count|
+-------+-----+
|     50|  583|
|    258|  509|
|    100|  508|
|    181|  507|
|    294|  485|
+-------+-----+
only showing top 5 rows



In [29]:
# Аналогичная сумма для худших 5 фильмов. Просто меняем порядок сортировки.
ratings_spark.groupby('item_id').sum('rating').sort('sum(rating)', ascending=True).show(5)

+-------+-----------+
|item_id|sum(rating)|
+-------+-----------+
|    830|          1|
|   1339|          1|
|   1493|          1|
|   1621|          1|
|   1618|          1|
+-------+-----------+
only showing top 5 rows



In [30]:
# Аналогичное количество оценок для худших 5 фильмов. Просто меняем порядок сортировки.
ratings_spark.groupby('item_id').count().sort('count', ascending=True).show(5)

+-------+-----+
|item_id|count|
+-------+-----+
|   1460|    1|
|   1618|    1|
|   1339|    1|
|   1645|    1|
|   1650|    1|
+-------+-----+
only showing top 5 rows

